In [79]:
import os
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import io
from scipy import stats
from scipy import ndimage
import emd

from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


# Helper function for the second level sift
def mask_sift_second_layer(IA, masks, config={}):
    imf2 = np.zeros((IA.shape[0], IA.shape[1], config['max_imfs']))
    for ii in range(IA.shape[1]):
        config['mask_freqs'] = masks[ii:]
        tmp = emd.sift.mask_sift(IA[:, ii], **config)
        imf2[:, ii, :tmp.shape[1]] = tmp
    return imf2

def get_holo_trl(filename, sys_type):
    raw=mne.io.read_raw_fif(filename)

    if sys_type == 'A':
        ag_num = [13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 46, 48, 56, 57, 58, 60, 61, 62, 63, 64, 74, 77, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 110, 112, 120, 121, 122, 123, 124, 125, 126, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160]

    if sys_type == 'B':
        ag_num = [9, 13, 15, 16, 17, 18, 20, 21, 22, 25, 26, 27, 29, 30, 31, 33, 34, 49, 50, 58, 59, 60, 61, 62, 63, 64, 76, 79, 81, 82, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 98, 110, 114, 121, 123, 124, 125, 126, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160]

    ag_names = ['AG%.3d'%s for s in ag_num]
    ag_picks = mne.pick_channels(raw.ch_names, ag_names)
    
    raw.pick(ag_picks)
    
    epochs = mne.make_fixed_length_epochs(raw, duration=2, preload=False)
    epochs.drop_bad(reject = dict(mag=3e-12))      # unit: T (magnetometers)

    psd, freq_ = mne.time_frequency.psd_array_welch(epochs.get_data() * 1e+15, 
                                                    epochs.info['sfreq'], 
                                                    fmin=8, fmax=12, 
                                                    n_fft=256, average='mean')
    psd_M=np.expand_dims(psd[0].mean(axis = 1),axis=1)
    psd_evk=mne.EvokedArray(psd_M,epochs.info,comment='raw_data')
    temp_n = None
    alpha_channels = []
    for i in range(5):
        psd_evk.pick('all',exclude=[temp_n])
        temp_n, temp_t = psd_evk.get_peak()
        alpha_channels.append(temp_n)
        pass

    print(alpha_channels)
    
    epochs.load_data()
    x = epochs.copy().pick_channels([alpha_channels[0]]).get_data()
    x = np.squeeze(x) * 1e+15

    n_trl = epochs.get_data().shape[0]
    sholo_temp = np.zeros((n_trl, 64, 64))

    sample_rate = np.int(raw.info['sfreq'])
    config_lay1 = emd.sift.get_config('mask_sift')
    config_lay1['max_imfs'] = 7
    config_lay1['mask_amp_mode'] = 'ratio_sig'
    config_lay1['imf_opts/sd_thresh'] = 0.05
    config_lay1['verbose'] = 'CRITICAL'

    config_lay2 = emd.sift.get_config('mask_sift')
    config_lay2['mask_amp_mode'] = 'ratio_sig'
    config_lay2['mask_amp'] = 2
    config_lay2['max_imfs'] = 5
    config_lay2['imf_opts/sd_thresh'] = 0.05
    config_lay2['envelope_opts/interp_method'] = 'mono_pchip'
    config_lay2['verbose'] = 'CRITICAL'
    # Carrier frequency histogram definition
    carrier_hist = (1, 100, 64, 'log')
    # AM frequency histogram definition
    am_hist = (1e-2, 32, 64, 'log')

    for n in range(n_trl):
        imf = emd.sift.mask_sift(x[n], **config_lay1)
        IP, IF, IA = emd.spectra.frequency_transform(imf, sample_rate, 'nht')
        masks = np.array([25/2**ii for ii in range(12)])/sample_rate

        # Sift the first 5 first level IMFs
        imf2 = emd.sift.mask_sift_second_layer(IA, masks, sift_args=config_lay2)

        IP2, IF2, IA2 = emd.spectra.frequency_transform(imf2, sample_rate, 'nht')
        fcarrier, fam, holo = emd.spectra.holospectrum(IF, IF2, IA2, carrier_hist, am_hist)
        sholo = ndimage.gaussian_filter(holo, 1)

        sholo_temp[n] = sholo
        pass

    sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))
    return sholo_m, fam, fcarrier

def holo_gray_images(sholo_m, fam, fcarrier):
    px = 1/plt.rcParams['figure.dpi']  # pixel in inches
    fig, ax = plt.subplots(figsize=(32*px, 32*px))
    fig.subplots_adjust(0,0,1,1)
    ax.pcolormesh(fam, fcarrier, stats.zscore(sholo_m), cmap='hot_r', shading='nearest', vmin = -2, vmax = 2)

    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.spines["left"].set_visible(False)
    ax.spines["bottom"].set_visible(False)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    img_buf = io.BytesIO()
    fig.savefig(img_buf, format='png')
    im = Image.open(img_buf, )
    imgGray = im.convert('L')
    mage_sequence = imgGray.getdata()
    image_array = np.array(mage_sequence)
    return image_array

In [2]:
%matplotlib qt

In [66]:
group = 'dementia'
path = '/Users/kevinhsu/Documents/D/00_datasets/biomag_2022/holo/train/m/'

s = 10
sys_type = 'A'

filename = 'de_hokuto_%s%d-raw.fif'%(group, s)

raw=mne.io.read_raw_fif(filename)

if sys_type == 'A':
    ag_num = [13, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 46, 48, 56, 57, 58, 60, 61, 62, 63, 64, 74, 77, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 110, 112, 120, 121, 122, 123, 124, 125, 126, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160]

if sys_type == 'B':
    ag_num = [9, 13, 15, 16, 17, 18, 20, 21, 22, 25, 26, 27, 29, 30, 31, 33, 34, 49, 50, 58, 59, 60, 61, 62, 63, 64, 76, 79, 81, 82, 83, 84, 85, 86, 87, 88, 90, 91, 92, 93, 94, 95, 96, 97, 98, 110, 114, 121, 123, 124, 125, 126, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160]

ag_names = ['AG%.3d'%s for s in ag_num]
ag_picks = mne.pick_channels(raw.ch_names, ag_names)

raw.pick(ag_picks)

sholo_m, fam, fcarrier = get_holo_trl(filename, sys_type)

['AG160', 'AG142', 'AG151', 'AG085', 'AG155']


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/3492669636.py:64: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])
/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/3492669636.py:98: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


In [77]:
plt.pcolormesh(fam, fcarrier, sholo_m, cmap='ocean_r', shading='nearest')
plt.title('Holospectrum')
plt.xlabel('AM Frequency (Hz)')
plt.show()
sholo_m.shape

(64, 64)

In [80]:
im_array = holo_gray_images(sholo_m, fam, fcarrier)

In [87]:
group = 'control'


sys_types = ['A','B','A','A','A','A','A','B','B','B','A',
             'A','A','B','A','A','B','B','B','B','A','A','B','A','A','A','A',
             'A','A','A','B','B','A','A','B','A','A','A','A','B','A','A','A',
             'A','A','A','A','A','B','A','A','A','A','A','A','A','B','A','A',
             'A','A','A','A','A','A','A','A','A','A','A','A','A','A','A','A',
             'A','B','B','B','A','A','A','A','A','A','A','A','B','A','B','A',
             'A','A','B','B','B','B','A','B','B']

excluded_control = []

holo_c = []
for i in range(100):
    s = i + 1
    sys_type = sys_types[i]
    try:
        filename = 'de_hokuto_%s%d-raw.fif'%(group, s)
        sholo_m, fam, fcarrier = get_holo_trl(filename, sys_type)
        im_array = holo_gray_images(sholo_m, fam, fcarrier)
        holo_c.append(im_array)
    except:
        print('no id %d'%s)
        excluded_control.append(s)
    pass
    


['AG151', 'AG135', 'AG155', 'AG142', 'AG157']


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:64: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])
/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:98: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


['AG151', 'AG154', 'AG152', 'AG155', 'AG143']


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:64: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])
/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:98: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


['AG064', 'AG019', 'AG018', 'AG060', 'AG154']


...

/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:64: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])
/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:98: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


['AG140', 'AG142', 'AG141', 'AG138', 'AG139']


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:64: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])
/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:98: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


['AG156', 'AG157', 'AG141', 'AG142', 'AG125']


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:64: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])
/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:98: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


In [88]:
group = 'dementia'


sys_types = ['A','B','A','B','A','B','B','B','B',
             'A','B','A','A','A','A','A','A','A',
             'A','B','A','B','A','A','B','A','A',
             'A','A']
excluded_dementia = []
holo_d = []
for i in range(29):
    s = i + 1
    sys_type = sys_types[i]
    try:
        filename = 'de_hokuto_%s%d-raw.fif'%(group, s)
        sholo_m, fam, fcarrier = get_holo_trl(filename, sys_type)
        im_array = holo_gray_images(sholo_m, fam, fcarrier)
        holo_d.append(im_array)
    except:
        print('no id %d'%s)
        excluded_dementia.append(s)
    pass
    

['AG154', 'AG152', 'AG145', 'AG138', 'AG150']


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:64: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])
/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:98: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


['AG083', 'AG160', 'AG157', 'AG082', 'AG156']


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:64: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])
/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:98: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


['AG021', 'AG025', 'AG013', 'AG017', 'AG019']


...

/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:64: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])
/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:98: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


no id 27
['AG133', 'AG054', 'AG134', 'AG051', 'AG052']


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:64: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])
/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:98: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


['AG021', 'AG059', 'AG063', 'AG017', 'AG046']


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:64: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])
/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:98: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


In [89]:

group = 'mci'


sys_types = ['B','B','B','B','B','B','B','B','B',
             'B','A','B','B','B','B']
excluded_mci = []
holo_m = []
for i in range(15):
    s = i + 1
    sys_type = sys_types[i]
    try:
        filename = 'de_hokuto_%s%d-raw.fif'%(group, s)
        sholo_m, fam, fcarrier = get_holo_trl(filename, sys_type)
        im_array = holo_gray_images(sholo_m, fam, fcarrier)
        holo_m.append(im_array)
    except:
        print('no id %d'%s)
        excluded_mci.append(s)
    pass
    

['AG101', 'AG100', 'AG105', 'AG104', 'AG122']


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:64: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])
/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:98: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


['AG101', 'AG027', 'AG141', 'AG142', 'AG139']


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:64: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])
/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:98: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


no id 3
['AG139', 'AG140', 'AG137', 'AG136', 'AG142']


...

/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:64: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])
/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:98: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


no id 13
['AG156', 'AG154', 'AG157', 'AG159', 'AG155']


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:64: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])


['AG060', 'AG059', 'AG057', 'AG061', 'AG035']


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:64: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])
/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_87452/526044833.py:98: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


In [90]:
print(excluded_control)
print(excluded_dementia)
print(excluded_mci)
holo_2d_d = np.vstack(holo_d)
holo_2d_c = np.vstack(holo_c)
holo_2d_m = np.vstack(holo_m)

holo_2d = np.concatenate((holo_2d_c, holo_2d_d, holo_2d_m),axis=0)

y = [0] * holo_2d_c.shape[0] + [1] * holo_2d_d.shape[0] + [2] * holo_2d_m.shape[0]
y = np.array(y)
print(holo_2d.shape)
print(len(y))

[49, 55, 60, 68, 90, 94, 97]
[6, 8, 12, 18, 25, 27]
[3, 6, 7, 11, 13]
(126, 2560)
126


In [3]:
X = holo_2d / 255.0


wclf = svm.SVC(kernel="linear", class_weight={0: 1, 1: 10, 2:1})
scores = cross_validate(wclf, X, y, cv=10, return_train_score=True)
print(scores['train_score'].mean())
print(scores['test_score'].mean())

1.0
0.825


In [12]:


clf = MLPClassifier(solver='adam', learning_rate_init=0.00001,
                    max_iter=20000, random_state=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)
clf.fit(X_train, y_train)
print("Training set score: %f" % clf.score(X_train, y_train))
print("Test set score: %f" % clf.score(X_test, y_test))


Training set score: 0.890000
Test set score: 0.884615


In [28]:
X = holo_2d / 255.0

clf = MLPClassifier(hidden_layer_sizes=(256, 128, 32, 16, 8), solver='adam', learning_rate_init=0.00001,
                    max_iter=20000, random_state=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)
clf.fit(X_train, y_train)
print("Training set score: %f" % clf.score(X_train, y_train))
print("Test set score: %f" % clf.score(X_test, y_test))


Training set score: 0.980000
Test set score: 0.923077


In [7]:
'''with open('feature_holo_1channel.npy', 'wb') as f:
    np.save(f, holo_2d)
    np.save(f, holo_2d_c)
    np.save(f, holo_2d_d)
    np.save(f, holo_2d_m)
    np.save(f, y)
'''

In [10]:
with open('feature_holo_1channel.npy', 'rb') as f:# originally, it is test.npy
    holo_2d = np.load(f)
    holo_2d_c = np.load(f)
    holo_2d_d = np.load(f)
    holo_2d_m = np.load(f)
    y = np.load(f)

In [5]:
len(y)

126

In [56]:
# prepare answer sheet

group = 'test'


sys_types = ['A','A','B','B','A','B','A','B','A',
             'A','B','A','A','A','A','B','B','A',
             'A','A','B','B','B','B','B','B','B',
             'B','B','B','A','A','B','B','A','A',
             'A','A','A','A','A','B']

excluded_test = []
holo_t = []
for i in range(42):
    s = i + 1
    sys_type = sys_types[i]
    try:
        filename = 'de_hokuto_%s%d-raw.fif'%(group, s)
        sholo_m, fam, fcarrier = get_holo_trl(filename, sys_type)
        im_array = holo_gray_images(sholo_m, fam, fcarrier)
        holo_t.append(im_array)
    except:
        print('no id %d'%s)
        excluded_test.append(s)
    pass
    

['AG083', 'AG156', 'AG062', 'AG124', 'AG140']


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_91705/1459849246.py:71: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])
/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_91705/1459849246.py:105: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


['AG141', 'AG159', 'AG139', 'AG088', 'AG160']


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_91705/1459849246.py:71: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])
/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_91705/1459849246.py:105: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


['AG121', 'AG134', 'AG150', 'AG123', 'AG139']


...

/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_91705/1459849246.py:71: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])
/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_91705/1459849246.py:105: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


['AG122', 'AG121', 'AG125', 'AG124', 'AG123']


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_91705/1459849246.py:71: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])
/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_91705/1459849246.py:105: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


['AG142', 'AG141', 'AG145', 'AG154', 'AG064']


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_91705/1459849246.py:71: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])
/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_91705/1459849246.py:105: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


In [58]:
print(excluded_test)
holo_2d_t = np.vstack(holo_t)

with open('feature_holo_1channel_test.npy', 'wb') as f:
    np.save(f, holo_2d_t)
    
holo_2d_t.shape

[16]


(41, 2560)

In [ ]:
clf.predict(holo_2d_t)

In [ ]:
clf.predict_proba(psd_median_t).round(3)

In [59]:
with open('feature_holo_1channel_test.npy', 'rb') as f:
    holo_2d_t = np.load(f)
    
holo_2d_t.shape

(41, 2560)

In [60]:
X_test = holo_2d_t / 255.0
clf.predict(X_test)

array([0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 2, 0, 1, 1, 1, 0, 2,
       0, 0, 0, 1, 0, 2, 0, 0, 1, 0, 2, 0, 0, 0, 1, 1, 0, 1, 2])

In [61]:
clf.predict_proba(X_test).round(3)

array([[0.688, 0.137, 0.175],
       [0.013, 0.417, 0.57 ],
       [0.994, 0.005, 0.   ],
       [0.608, 0.247, 0.145],
       [0.944, 0.033, 0.023],
       [0.911, 0.037, 0.052],
       [0.975, 0.023, 0.002],
       [0.781, 0.095, 0.124],
       [0.001, 0.999, 0.   ],
       [0.995, 0.005, 0.   ],
       [0.58 , 0.178, 0.242],
       [0.118, 0.866, 0.016],
       [0.998, 0.002, 0.   ],
       [0.   , 0.998, 0.002],
       [0.001, 0.997, 0.002],
       [0.097, 0.329, 0.574],
       [0.766, 0.085, 0.149],
       [0.001, 0.998, 0.001],
       [0.   , 1.   , 0.   ],
       [0.   , 0.955, 0.045],
       [0.989, 0.011, 0.001],
       [0.093, 0.205, 0.701],
       [0.975, 0.013, 0.012],
       [0.991, 0.009, 0.001],
       [0.811, 0.175, 0.015],
       [0.216, 0.514, 0.27 ],
       [0.967, 0.02 , 0.013],
       [0.061, 0.304, 0.635],
       [0.426, 0.218, 0.356],
       [0.589, 0.118, 0.293],
       [0.009, 0.981, 0.01 ],
       [0.999, 0.001, 0.   ],
       [0.008, 0.392, 0.6  ],
       [0.

In [63]:
y_test = clf.predict(X_test)
y_prob = clf.predict_proba(X_test).round(3)

labels = 'control', 'dementia','mci'
for yi, yj in zip(y_test, y_prob):
    print('%s, %f'%(labels[yi], np.max(yj)))
    pass


control, 0.688000
mci, 0.570000
control, 0.994000
control, 0.608000
control, 0.944000
control, 0.911000
control, 0.975000
control, 0.781000
dementia, 0.999000
control, 0.995000
control, 0.580000
dementia, 0.866000
control, 0.998000
dementia, 0.998000
dementia, 0.997000
mci, 0.574000
control, 0.766000
dementia, 0.998000
dementia, 1.000000
dementia, 0.955000
control, 0.989000
mci, 0.701000
control, 0.975000
control, 0.991000
control, 0.811000
dementia, 0.514000
control, 0.967000
mci, 0.635000
control, 0.426000
control, 0.589000
dementia, 0.981000
control, 0.999000
mci, 0.600000
control, 0.998000
control, 0.963000
control, 0.997000
dementia, 0.981000
dementia, 1.000000
control, 0.998000
dementia, 0.994000
mci, 0.881000


In [74]:
filename = 'de_hokuto_%s%d_old-raw.fif'%(group, 16)
sys_type = "B"
sholo_m, fam, fcarrier = get_holo_trl(filename, sys_type)
im_array = holo_gray_images(sholo_m, fam, fcarrier)

x_temp =np.expand_dims(im_array, axis=0)
print(clf.predict(x_temp/255.0))
print(clf.predict_proba(x_temp/255.0).round(3))

['AG091', 'AG088', 'AG063', 'AG083', 'AG092']


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_91705/4273186858.py:71: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])


[2]
[[0.003 0.235 0.762]]


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_91705/4273186858.py:105: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


In [46]:

filename = 'de_hokuto_%s%d-raw.fif'%(group, 21)
sys_type = "B"
sholo_m, fam, fcarrier = get_holo_trl(filename, sys_type)
im_array = holo_gray_images(sholo_m, fam, fcarrier)

x_temp =np.expand_dims(im_array, axis=0)
print(clf.predict(x_temp/255.0))
print(clf.predict_proba(x_temp/255.0).round(3))

['AG150', 'AG131', 'AG132', 'AG146', 'AG121']


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_91705/1877454791.py:75: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])


[1]
[[0.    0.955 0.045]]


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_91705/1877454791.py:109: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


In [47]:
filename = 'de_hokuto_%s%d-raw.fif'%(group, 29)
sys_type = "B"
sholo_m, fam, fcarrier = get_holo_trl(filename, sys_type)
im_array = holo_gray_images(sholo_m, fam, fcarrier)

x_temp =np.expand_dims(im_array, axis=0)
print(clf.predict(x_temp/255.0))
print(clf.predict_proba(x_temp/255.0).round(3))

['AG143', 'AG145', 'AG140', 'AG158', 'AG154']


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_91705/1877454791.py:75: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])


[2]
[[0.061 0.304 0.635]]


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_91705/1877454791.py:109: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


In [48]:
filename = 'de_hokuto_%s%d-raw.fif'%(group, 32)
sys_type = "A"
sholo_m, fam, fcarrier = get_holo_trl(filename, sys_type)
im_array = holo_gray_images(sholo_m, fam, fcarrier)

x_temp =np.expand_dims(im_array, axis=0)
print(clf.predict(x_temp/255.0))
print(clf.predict_proba(x_temp/255.0).round(3))

['AG057', 'AG020', 'AG147', 'AG063', 'AG058']


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_91705/1877454791.py:75: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])


[1]
[[0.009 0.981 0.01 ]]


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_91705/1877454791.py:109: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))


In [76]:
#[6, 16, 21, 29, 32, 34, 38, 39]
filename = 'de_hokuto_%s%d-raw.fif'%(group, 38)
sys_type = "A"
sholo_m, fam, fcarrier = get_holo_trl(filename, sys_type)
im_array = holo_gray_images(sholo_m, fam, fcarrier)

# test 6
x_temp =np.expand_dims(im_array, axis=0)
print(clf.predict(x_temp/255.0))
print(clf.predict_proba(x_temp/255.0).round(3))

['AG141', 'AG140', 'AG139', 'AG154', 'AG152']


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_91705/1508224151.py:71: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sample_rate = np.int(raw.info['sfreq'])


[1]
[[0.004 0.994 0.002]]


/var/folders/7w/cb7mhl253wdbc67t80fs_bmc0000gn/T/ipykernel_91705/1508224151.py:105: RuntimeWarning: Mean of empty slice
  sholo_m = np.squeeze(np.nanmean(sholo_temp, axis = 0))
